# Damped bead on a rotating Hoop

From Sec. 3.5 in Strogatz.  Full equation:
$$
m R \ddot{\phi} = -b \dot{\phi} - m g \sin \phi + m R^2 \omega \sin \phi \cos \phi
$$

If we introduce nondimensionalizations
$$
\gamma = \frac{R \omega^2}{g} \hspace{2cm} \epsilon = \frac{m^2 g R}{b^2}  \hspace{2cm} \tau = \frac{t}{T} = \frac{b}{mg},
$$
the equation becomes
$$
\epsilon \frac{d^2 \phi}{d \tau^2} = - \frac{d \phi}{d \tau} - \sin \phi + \gamma \sin \phi \cos \phi.
$$
For $\epsilon \ll 1$ and $\gamma = \mathcal{O}(1)$, the system is overdamped and approximately first-order.  The system undergoes a pitchfork bifurcation at $\gamma = 1$.

In [7]:
# Matplotlib
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rcParams
mpl.rc('text', usetex=True)
mpl.rc('font', family='serif')
mpl.rc('xtick', labelsize=14)
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', labelsize=20)
mpl.rc('axes', titlesize=20)
mpl.rc('figure', figsize=(6, 4))
%config InlineBackend.figure_format = 'retina'

from numpy.linalg import matrix_rank
import sys
sys.path.append('../src')
sys.path.append('../solvers')
from rotating_hoop import RotatingHoop
from learning import KRidgeReg, NeuralNet, BuckyNet, KRidgeReg_struct
from nullspace_search import get_nondim_numbers, fit_allnondim
from helper_functions import prettify_results


## Get Rotating hoop data

In [ ]:
nsamples = int(1e3)
output_type = 'svd' # options: 'dynamic', 'static', 'svd' - 
num_nondim = 2 # 
num_modes = 4
tsteps = 500 # For 'dynamic', use smaller number of tsteps - dynamic still takes too long
tend = 100
phi_init = [1, 0]

## Get solution
R = RotatingHoop(nsamples=nsamples, output_type=output_type, modes=num_modes, time_steps=tsteps, tend=tend, phi0=phi_init)
inputs, outputs = R.get_data()
dim_matrix, names = R.get_dim_matrix(include_names=True)

## In the static case, not all input parameters are used
# In the SVD case 
if output_type == 'static':
    num_nondim = 1
else:
    num_nondim = inputs.shape[1] - matrix_rank(dim_matrix)


print(num_nondim)

## Get non-dimensional numbers from Kernel Ridge Fit

In [9]:
## Consider hyperparameter optimization over l1_reg and alpha
l1_reg = 1e-3 ## Solution is sensitive to L1
alpha = 1e-4
kernel = 'rbf'
gamma = 30
use_test_set = True
normalize=False
num_trials = 5
tol = 0.1
max_denominator = 10

# Ridge Regression
K = KRidgeReg(inputs, outputs, dim_matrix, num_nondim=num_nondim, #normalize=normalize,
 l1_reg=l1_reg, alpha=alpha, kernel=kernel, gamma=gamma)



In [10]:
print('\n NOT using test set')
K.use_test_set = False
x = K.multi_run(ntrials=num_trials)


print(R.get_true_nondim())
pi1 = x[:, 0]/x[1, 0]
prettify_results(pi1, names, tol=tol, max_degree=max_denominator)
if output_type != 'static':
    pi2 = x[:, 1]/x[1, 1]
    prettify_results(pi2, names, tol=tol, max_degree=max_denominator)


[[ 0  1  0 -1  2]
 [ 2  1 -2  1  0]]
[ 2.0080266  1.        -2.0080266  1.0080266 -0.0080266]
m : 2
R : 1
b : -2
g : 1
w : 0


<IPython.core.display.Math object>

[ 0.05088969  1.         -0.05088969 -0.94911031  1.94911031]
m : 0
R : 1
b : 0
g : -1
w : 2


<IPython.core.display.Math object>

In [11]:
print('using test set')
K.use_test_set = True
x2 = K.multi_run(ntrials=num_trials)


print(R.get_true_nondim())
pi1 = x[:, 0]/x[1, 0]
prettify_results(pi1, names, tol=tol, max_degree=max_denominator)
if output_type != 'static':
    pi2 = x[:, 1]/x[1, 1]
    prettify_results(pi2, names, tol=tol, max_degree=max_denominator)

using test set
2.4931829344566285
1.925358442348529
[[ 0  1  0 -1  2]
 [ 2  1 -2  1  0]]
[ 2.0080266  1.        -2.0080266  1.0080266 -0.0080266]
m : 2
R : 1
b : -2
g : 1
w : 0


<IPython.core.display.Math object>

[ 0.05088969  1.         -0.05088969 -0.94911031  1.94911031]
m : 0
R : 1
b : 0
g : -1
w : 2


<IPython.core.display.Math object>

In [12]:

print('\n Normalize data')
K.normalize()
K.use_test_set = True
x3 = K.multi_run(ntrials=num_trials)


print(R.get_true_nondim())
pi1 = x[:, 0]/x[1, 0]
prettify_results(pi1, names, tol=tol, max_degree=max_denominator)
if output_type != 'static':
    pi2 = x[:, 1]/x[1, 1]
    prettify_results(pi2, names, tol=tol, max_degree=max_denominator)
## Normalizing seems like a bad idea..


 Normalize data
2.221002182479567
1.6408448482579319
1.1344045742885176
1.080364151740218
[[ 0  1  0 -1  2]
 [ 2  1 -2  1  0]]
[ 2.0080266  1.        -2.0080266  1.0080266 -0.0080266]
m : 2
R : 1
b : -2
g : 1
w : 0


<IPython.core.display.Math object>

[ 0.05088969  1.         -0.05088969 -0.94911031  1.94911031]
m : 0
R : 1
b : 0
g : -1
w : 2


<IPython.core.display.Math object>